In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input/100-bird-species/285 birds'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt

import os
import warnings
warnings.simplefilter('ignore')

import cv2
import tensorflow as tf
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Dense,Conv2D,Lambda,Flatten
from tensorflow.keras.layers import Input,MaxPooling2D

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input

from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator


from keras.models import load_model
from keras.preprocessing.image import load_img,img_to_array

In [ ]:
train = '../input/100-bird-species/285 birds/train'
test = '../input/100-bird-species/285 birds/test'
val = '../input/100-bird-species/285 birds/valid'
df = pd.read_csv("../input/100-bird-species/285 birds/Bird Species.csv")

In [ ]:
print("Total Number Of species are: ",len(os.listdir(train)))

In [ ]:
count = 0
for i in os.listdir(train):
    x = train +'/'+ str(i)
    count = count + len(x)
print("Total No of Images",count)

In [ ]:
for i in os.listdir(train):
    x = train +'/'+ str(i)
    print("Total No of Images in {} are {}".format(str(i),len(x)))

In [ ]:
df.head()

In [ ]:
df = df.sample(frac=1)
df_for_img = df.head()

In [ ]:
paths = []
for i in range(5):
    x = df_for_img.iloc[i][0].split('\\')
    y = '/'.join(x)
    paths.append(y)
df_path = pd.DataFrame(paths)
df_path.columns=['paths']
df_path

In [ ]:
plt.figure(figsize=(16,11))
folder = '../input/100-bird-species/285 birds/'    
for i in range(5):
    plt.subplot(1,5,i+1)
    x = folder + df_path.iloc[i][0]
    image = plt.imread(x)
    plt.imshow(image)
    plt.title(df_for_img.iloc[i][1])

In [ ]:
image_size = [224,224]
vgg = VGG16(input_shape = image_size + [3],
            weights = 'imagenet',include_top=False)

for layer in vgg.layers:
    layer.trainable = False

In [ ]:
folder = len(os.listdir(train))
folder

In [ ]:
x = Flatten()(vgg.output)
classification = Dense(int(folder),activation='softmax')(x)

model = Model(inputs = vgg.input,outputs = classification)
model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam',
             metrics = ['accuracy'])

In [ ]:
train_generator = ImageDataGenerator(rescale = 1./255,
                                    shear_range = 0.2,
                                    zoom_range = 0.2,
                                    horizontal_flip = True,
                                    width_shift_range=0.2,
                                    height_shift_range=0.2)

test_generator = ImageDataGenerator(rescale=1./255)

In [ ]:
train_set = train_generator.flow_from_directory(train,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode = 'categorical')

valid_set = train_generator.flow_from_directory(val,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode = 'categorical')

test_set = test_generator.flow_from_directory(test,
                                               target_size=(224,224),
                                               batch_size=32,
                                               class_mode = 'categorical')

In [ ]:
if tf.test.gpu_device_name(): 
    print('Default GPU Device:{}'.format(tf.test.gpu_device_name()))
else:
    print("Please install GPU version of TF")

In [ ]:
!nvidia-smi

In [ ]:
history = model.fit(train_set,validation_data = valid_set,
                    epochs=50,steps_per_epoch=len(train_set),
                    validation_steps=len(test_set))

In [ ]:
#Training
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title("Model Accuracy")
plt.ylabel("Accuracy")
plt.xlabel("Epoch")
plt.legend(['Train','Test'],loc='upper left')
plt.show()
#Loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title("Model Loss")
plt.ylabel("Loss")
plt.xlabel("Epoch")
plt.legend(['Train','Test'],loc='upper left')
plt.show()

# Model Saving

In [ ]:
model.save("birds_classification_vgg.h5")
loaded_model = load_model('birds_classification_vgg.h5', compile = False)

In [ ]:
model.save("./saved_model/")

## Converting to tflite
without quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
tflite_model = converter.convert()

In [ ]:
len(tflite_model)

## With quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_saved_model("./saved_model")
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

In [ ]:
len(tflite_quant_model)

In [ ]:
with open("tflite_model.tflite", "wb") as f:
    f.write(tflite_model)

In [ ]:
with open("tflite_quant_model.tflite", "wb") as f:
    f.write(tflite_quant_model)

## Testing

In [ ]:
labels = train_set.class_indices
labels_reverse = {k:v for v,k in labels.items()}
labels_reverse

In [ ]:
def output(location):
    img = load_img(location, target_size = (224, 224, 3))
    img = img_to_array(img)
    img = img / 255
    img = np.expand_dims(img, [0])
    answer = loaded_model.predict(img)
    y_class = answer.argmax(axis = -1)
    y = " ".join(str(x) for x in y_class)
    y = int(y)
    res = labels_reverse[y]
    return res

In [ ]:
img = '../input/100-bird-species/birds/valid/AMERICAN BITTERN/1.jpg'
pic = load_img(img, target_size = (224, 224, 3))
plt.imshow(pic)
output(img)

In [ ]:
img = '../input/100-bird-species/birds/valid/AMERICAN COOT/1.jpg'
pic = load_img(img, target_size = (224, 224, 3))
plt.imshow(pic)
output(img)

In [ ]:
model.evaluate(test_set)